# MatSim specific validation

This page goes through validation checks available in GeNet. Available as a jupyter notebook or wiki page.

You can generate a validation report for the genet Network encompassing validity of the network, schedule and routing (of the transit services in the schedule on the network). It aims to provide a good collection of checks known to have affected MatSim simulations in the past. The report is a simple dictionary with keys: `graph`, `schedule` and `routing`.

In [1]:
# read sample network
from genet import read_matsim
import os

path_to_matsim_network = '../example_data/pt2matsim_network'

network = os.path.join(path_to_matsim_network, 'network.xml')
schedule = os.path.join(path_to_matsim_network, 'schedule.xml')
vehicles = os.path.join(path_to_matsim_network, 'vehicles.xml')
n = read_matsim(
    path_to_network=network, 
    epsg='epsg:27700', 
    path_to_schedule=schedule, 
    path_to_vehicles=vehicles
)
# you don't need to read the vehicles file, but doing so ensures all vehicles
# in the schedule are of the expected type and the definition of the vehicle
# is preserved
n.print()

Graph info: Name: 
Type: MultiDiGraph
Number of nodes: 1662
Number of edges: 3166
Average in degree:   1.9049
Average out degree:   1.9049 
Schedule info: Schedule:
Number of services: 9
Number of routes: 68
Number of stops: 118


In [2]:
report = n.generate_validation_report()

2022-07-14 16:19:28,787 - Checking validity of the Network
2022-07-14 16:19:28,794 - Checking validity of the Network graph
2022-07-14 16:19:28,795 - Checking network connectivity for mode: car
2022-07-14 16:19:29,009 - Checking network connectivity for mode: walk
2022-07-14 16:19:29,027 - Checking network connectivity for mode: bike
2022-07-14 16:19:29,436 - Checking validity of the Schedule
2022-07-14 16:19:29,514 - 11728 unused vehicles have been found.
2022-07-14 16:19:29,546 - No vehicles being used for multiple trips have been found.


The `graph` section describes strongly connected components of the modal subgraphs, for modes that agents in MATSim need to find routes on: `car`, and `walk` and `bike` if using the `multimodal.contrib`. In addition to this, it also flags links of length 1km or longer that can be inspected separately.

In [3]:
from pprint import pprint
pprint(report['graph'])

{'graph_connectivity': {'bike': {'number_of_connected_subgraphs': 0,
                                 'problem_nodes': {'dead_ends': [],
                                                   'unreachable_node': []}},
                        'car': {'number_of_connected_subgraphs': 1,
                                'problem_nodes': {'dead_ends': [],
                                                  'unreachable_node': []}},
                        'walk': {'number_of_connected_subgraphs': 0,
                                 'problem_nodes': {'dead_ends': [],
                                                   'unreachable_node': []}}},
 'link_attributes': {'links_over_1km_length': {'link_ids': [],
                                               'number_of': 0,
                                               'percentage': 0.0},
                     'zero_attributes': {}}}


The `schedule` section describes correctness of the schedule on three levels:
    
- `schedule_level`: Overall look at the schedule validity. A `Schedule` is valid if:
    - all of its' services are valid
    - its' services are uniquely indexed
    
    Schedule `has_valid_services` if all services within the schedule are deemed valid. The invalid services are 
    flagged in `invalid_services` and the invalid stages of schedule validity are flagged in `invalid_stages`.
- `service_level`: Provides a look at validity of services within the schedule. It is indexed by service ids. Each
`Service` is valid if:
    - each of its' routes is valid
    - its' routes are uniquely indexed
    
    A service `has_valid_routes` if all routes within the service are deemed valid. The invalid routes are 
    flagged in `invalid_routes` and the invalid stages of service validity are flagged in `invalid_stages`.
- `route_level`: Provides a look at validity of each route within each service indexed by service id and route id
(or service id and the indexin the `Service.routes` list if not uniquely indexed). Each `Route` is valid if it
    - has more than one `Stop`
    - has correctly ordered route (the stops (their link reference ids) and links a route refers to are in the same 
    order)
    - arrival and departure offsets are correct (each stop has one and they are correctly ordered temporally)
    - does not have self loops (there are no trips such as: Stop A -> Stop A)
    
    If a route satisfies the above `is_valid_route` is `True`. If not, the `invalid_stages` flag where the route
    did not satisfy validity conditions.
    
    (Nb. The same dictionary can be generated by using `Schedule` object's own `generate_validation_report` method.)

- `vehicle_level`: Looks at the validity of vehicle definitions and their uses. Checks that there is a valid definition for each vehicle, which consists of the following components:
    - whether any definitions are missing
    - whether any vehicles are not being used any more
    - whether any vehicles are being used for multiple trips

In [4]:
pprint(report['schedule']['schedule_level'])

{'has_valid_services': True,
 'invalid_services': [],
 'invalid_stages': [],
 'is_valid_schedule': True}


In [5]:
pprint(report['schedule']['service_level']['12430'])

{'has_valid_routes': True,
 'invalid_routes': [],
 'invalid_stages': [],
 'is_valid_service': True}


In [6]:
pprint(report['schedule']['route_level']['12430'])

{'VJ06420fdab0dfe5c8e7f2f9504df05cf6289cd7d3': {'invalid_stages': [],
                                                'is_valid_route': True},
 'VJ06cd41dcd58d947097df4a8f33234ef423210154': {'invalid_stages': [],
                                                'is_valid_route': True},
 'VJ0f3c08222de16c2e278be0a1bf0f9ea47370774e': {'invalid_stages': [],
                                                'is_valid_route': True},
 'VJ15419796737689e742962a625abcf3fd5b3d58b1': {'invalid_stages': [],
                                                'is_valid_route': True},
 'VJ235c8fca539cf931b3c673f9b056606384aff950': {'invalid_stages': [],
                                                'is_valid_route': True},
 'VJ8f9aea7491080b0137d3092706f53dc11f7dba45': {'invalid_stages': [],
                                                'is_valid_route': True},
 'VJ948e8caa0f08b9c6bf6330927893942c474b5100': {'invalid_stages': [],
                                                'is_valid_route': True},

In [7]:
pprint(report['schedule']['vehicle_level'])

{'vehicle_definitions_valid': True,
 'vehicle_definitions_validity_components': {'missing_vehicles': {'missing_vehicles_types': set(),
                                                                  'vehicles_affected': {}},
                                             'multiple_use_vehicles': {},
                                             'unused_vehicles': {'veh_0_bus',
                                                                 'veh_10000_subway',
                                                                 'veh_10001_subway',
                                                                 'veh_10002_subway',
                                                                 'veh_10003_subway',
                                                                 'veh_10004_subway',
                                                                 'veh_10005_subway',
                                                                 'veh_10006_subway',
                          

                                                                 'veh_11256_subway',
                                                                 'veh_11257_subway',
                                                                 'veh_11258_subway',
                                                                 'veh_11259_subway',
                                                                 'veh_11260_subway',
                                                                 'veh_11261_subway',
                                                                 'veh_11262_subway',
                                                                 'veh_11263_subway',
                                                                 'veh_11264_subway',
                                                                 'veh_11265_subway',
                                                                 'veh_11266_subway',
                                                                 

                                                                 'veh_11731_bus',
                                                                 'veh_11732_bus',
                                                                 'veh_11733_bus',
                                                                 'veh_11734_bus',
                                                                 'veh_11735_bus',
                                                                 'veh_11736_bus',
                                                                 'veh_11737_bus',
                                                                 'veh_11738_bus',
                                                                 'veh_11739_bus',
                                                                 'veh_1173_bus',
                                                                 'veh_11740_bus',
                                                                 'veh_11741_bus',
                 

                                                                 'veh_12630_bus',
                                                                 'veh_12631_bus',
                                                                 'veh_12632_bus',
                                                                 'veh_12633_bus',
                                                                 'veh_12634_bus',
                                                                 'veh_12635_bus',
                                                                 'veh_12636_bus',
                                                                 'veh_12637_bus',
                                                                 'veh_12638_bus',
                                                                 'veh_12639_bus',
                                                                 'veh_1263_bus',
                                                                 'veh_12640_bus',
                 

                                                                 'veh_13080_bus',
                                                                 'veh_13081_bus',
                                                                 'veh_13082_bus',
                                                                 'veh_13083_bus',
                                                                 'veh_13084_bus',
                                                                 'veh_13085_bus',
                                                                 'veh_13086_bus',
                                                                 'veh_13087_bus',
                                                                 'veh_13088_bus',
                                                                 'veh_13089_bus',
                                                                 'veh_1308_bus',
                                                                 'veh_13090_bus',
                 

...

                                                                 'veh_4879_bus',
                                                                 'veh_4880_bus',
                                                                 'veh_4881_bus',
                                                                 'veh_4882_bus',
                                                                 'veh_4883_bus',
                                                                 'veh_4884_bus',
                                                                 'veh_4885_bus',
                                                                 'veh_4886_bus',
                                                                 'veh_4887_bus',
                                                                 'veh_4888_bus',
                                                                 'veh_4889_bus',
                                                                 'veh_4890_bus',
                            

                                                                 'veh_5587_bus',
                                                                 'veh_5588_bus',
                                                                 'veh_5589_bus',
                                                                 'veh_5590_bus',
                                                                 'veh_5591_bus',
                                                                 'veh_5592_bus',
                                                                 'veh_5593_bus',
                                                                 'veh_5594_bus',
                                                                 'veh_5595_bus',
                                                                 'veh_5596_bus',
                                                                 'veh_5597_bus',
                                                                 'veh_5598_bus',
                            

                                                                 'veh_6851_bus',
                                                                 'veh_6852_bus',
                                                                 'veh_6853_bus',
                                                                 'veh_6854_bus',
                                                                 'veh_6855_bus',
                                                                 'veh_6856_bus',
                                                                 'veh_6857_bus',
                                                                 'veh_6858_bus',
                                                                 'veh_6859_bus',
                                                                 'veh_6860_bus',
                                                                 'veh_6861_bus',
                                                                 'veh_6862_bus',
                            

                                                                 'veh_8058_subway',
                                                                 'veh_8059_subway',
                                                                 'veh_8060_subway',
                                                                 'veh_8061_subway',
                                                                 'veh_8062_subway',
                                                                 'veh_8063_subway',
                                                                 'veh_8064_subway',
                                                                 'veh_8065_subway',
                                                                 'veh_8066_subway',
                                                                 'veh_8067_subway',
                                                                 'veh_8068_subway',
                                                                 'veh_8069_s

                                                                 'veh_9541_subway',
                                                                 'veh_9542_subway',
                                                                 'veh_9543_subway',
                                                                 'veh_9544_subway',
                                                                 'veh_9545_subway',
                                                                 'veh_9546_subway',
                                                                 'veh_9547_subway',
                                                                 'veh_9548_subway',
                                                                 'veh_9549_subway',
                                                                 'veh_9550_subway',
                                                                 'veh_9551_subway',
                                                                 'veh_9552_s

Finally, the `routing` section describes routing of the transit schedule services onto the network graph.
- `services_have_routes_in_the_graph`: all routes have network routes and the links they refer to exist in the graph,
are connected (to nodes of preceding link is the from node of the next link in the chain) and the `modes` saved on the
link data accept the mode of the route.
- `service_routes_with_invalid_network_route`: flags routes not satifying the above,
- `route_to_crow_fly_ratio`: gives ratio of the length of route to crow-fly distance between each of the stops along 
route. If the route is invalid, it will result in 0. If the route has only one stop it will result in 
`'Division by zero'`.

In [8]:
pprint(report['routing'])

{'route_to_crow_fly_ratio': {'12430': {'VJ06420fdab0dfe5c8e7f2f9504df05cf6289cd7d3': 1.3239965103249995,
                                       'VJ06cd41dcd58d947097df4a8f33234ef423210154': 1.3239965103249995,
                                       'VJ0f3c08222de16c2e278be0a1bf0f9ea47370774e': 1.070199597773601,
                                       'VJ15419796737689e742962a625abcf3fd5b3d58b1': 1.3239965103249995,
                                       'VJ235c8fca539cf931b3c673f9b056606384aff950': 1.070199597773601,
                                       'VJ8f9aea7491080b0137d3092706f53dc11f7dba45': 1.070199597773601,
                                       'VJ948e8caa0f08b9c6bf6330927893942c474b5100': 1.070199597773601,
                                       'VJ95b4c534d7c903d76ec0340025aa88b81dba3ce4': 1.070199597773601,
                                       'VJeae6e634f8479e0b6712780d5728f0afca964e64': 1.3239965103249995,
                                       'VJeb72539d69ddf8e29f

The above report relies on a lot of convenience methods which can be used on their own. For example, you can list all invalid routes for the network using:

In [9]:
n.invalid_network_routes()

[]

In [10]:
n.schedule.is_valid_schedule()

True

Something that is not included in the validity report (because MATSim doesn't insist on it being satified) is strong connectivity of PT. You can call `is_strongly_connected` on `Schedule` or the schedule components: `Service` and `Route`. The process uses an underlying  directed graph of stop connections (which you can access by calling `graph` method on a schedule-type element, e.g. if `s` is a `genet.Service` object, `s.graph()` will give you this directed graph)).

In [11]:
n.schedule.is_strongly_connected()

False

In [12]:
n.schedule.graph().is_directed()

True